In [1]:
import sys
import numpy as np
import scipy
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import math
from datetime import datetime, timedelta, time, date

from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
%matplotlib inline

In [2]:
#using KNN to predict
train_path_1 = '../dataset/training/trajectories(table 5)_training.csv'
train_path_2 = '../dataset/dataSet_phase2/trajectories(table_5)_training2.csv'
test_path = '../dataset/dataSet_phase2/trajectories(table 5)_test2.csv'

train_df_1 = pd.read_csv(train_path_1)
train_df_2 = pd.read_csv(train_path_2)
train_df = train_df_1.append(train_df_2, ignore_index=True)

test_df = pd.read_csv(test_path)
train_df.starting_time = pd.to_datetime(train_df.starting_time)
test_df.starting_time = pd.to_datetime(test_df.starting_time)

In [582]:
# from 9-19 to 10-17 (except holiday)
NUM_TRAIN_DAYS = 20

# from 10-18 to 10-24
NUM_TSET_DAYS = 7

# define Holiday
NATIONNAL_START = date(2016,10,1)
NATIONNAL_END = date(2016,10,9)

MID_AUTUMN_START = date(2016,9,15)
MID_AUTUMN_END = date(2016,9,18)

TRAIN_START_DAY = date(2016,7,19)
#TRAIN_START_DAY = date(2016,10,10)
TRAIN_END_DAY = date(2016,10,17)

VALI_START_DAY = date(2016,10,11)
VALI_END_DAY = date(2016,10,17)

TEST_START_DAY = date(2016,10,18)
TEST_END_DAY = date(2016,10,24)

# 每条路劲的link
LINK_A_2 = [110,123,107,108,120,117]
LINK_A_3 = [110,123,107,108,119,114,118,122]
LINK_B_1 = [105,100,111,103,116,101,121,106,113]
LINK_B_3 = [105,100,111,103,122]
LINK_C_1 = [115,102,109,104,112,111,103,116,101,121,106,113]
LINK_C_3 = [115,102,109,104,112,111,103,122]

#每条路的link个数
LINK_LENGTH = {'table_A_2':6, 'table_A_3':8,'table_B_1':9,'table_B_3':5,'table_C_1':12,'table_C_3':8}               
# LINK_LENGTH = {'table_C_3':8}

inter_toll = [('A',2), ('A',3), ('B',1), ('B',3), ('C',1), ('C',3)]
# inter_toll = [('C',3)]


In [3]:
print(train_df.head(5))

  intersection_id  tollgate_id  vehicle_id       starting_time  \
0               B            3     1065642 2016-07-19 00:14:24   
1               B            3     1047198 2016-07-19 00:35:56   
2               B            1     1086390 2016-07-19 00:37:15   
3               A            2     1071181 2016-07-19 00:37:59   
4               B            1     1065807 2016-07-19 00:56:21   

                                          travel_seq  travel_time  
0  105#2016-07-19 00:14:24#9.56;100#2016-07-19 00...        70.85  
1  105#2016-07-19 00:35:56#11.58;100#2016-07-19 0...       148.79  
2  105#2016-07-19 00:37:15#5.26;100#2016-07-19 00...        79.76  
3  110#2016-07-19 00:37:59#13.74;123#2016-07-19 0...        58.05  
4  105#2016-07-19 00:56:21#16.08;100#2016-07-19 0...       137.98  


In [584]:
def MAPE(pred, true):
    return abs((true - pred) / true)

def cal_mape(pred_values, true_values):
    mape_mean = 0.0
    for i in range(len(pred_values)):
        pred_i = pred_values[i]
        true_i = true_values[i]
        mape_mean += abs((pred_i-true_i) / true_i)
    mape_mean /= len(pred_values)
    return mape_mean

def per_20min(dt):
    minute = int(math.floor(dt.minute / 20) * 20)
    second = 0
    dt_new = datetime(dt.year, dt.month, dt.day, dt.hour,minute, 0)
    return dt_new

# 9~19～10.17只有国庆节，因此只考虑国庆节
def remove_holiday(df):
    day_all = df.starting_time.dt.date
    df = df.loc[((day_all < MID_AUTUMN_START) | (day_all > MID_AUTUMN_END))]
    df = df.loc[((day_all < NATIONNAL_START) | (day_all > NATIONNAL_END))]
    return df


def slice_am_pm(df):
    hours = df.starting_time.dt.hour
    df_am = df.loc[(hours < 12)]
    df_pm = df.loc[(hours >= 12)]
    return df_am, df_pm

def slice_time(df):
    hour = df.starting_time.dt.hour  
    df_prev2h = df.loc[(((hour >= 6) & (hour < 8)) | ((hour >= 15) & (hour < 17)))]
    df_follow2h = df.loc[(((hour >= 8) & (hour < 10)) | ((hour >= 17) & (hour < 19)))]
    return df_prev2h, df_follow2h


def select_time(df):
    df['starting_time'] = df.starting_time.apply(
        per_20min)
    if {'vehicle_id','travel_seq'}.issubset(df.columns):
        df = df.drop(['vehicle_id','travel_seq'], axis=1)
    df = df.groupby(['intersection_id', 'tollgate_id', 'starting_time']).mean()
    df = df.reset_index()
    df = df.rename_axis({'travel_time':'avg_travel_time'}, axis='columns')
    hour = df.starting_time.dt.hour
    df = df.loc[((hour >= 6) & (hour < 10)) 
                     | ((hour >= 15) & (hour < 19))]
    return df

#创建6个表,把df数据按inter_toll分成6份放进set_table
def createTable(df): 
    set_table_fun = {}
    for i in range(len(inter_toll)):
        inter, toll = inter_toll[i]
        table_name = 'table_'+str(inter)+'_'+str(toll)
        set_table_fun[str(table_name)] = df.loc[(df.intersection_id == inter) & (df.tollgate_id == toll)]
    return set_table_fun

def getLink_avgTime(ser):  #ser是Series
    str_travel_seq = ser.split(';')
    link_time = []
    for i in range(len(str_travel_seq)):
        link_time.append(str_travel_seq[i].split('#')[-1])
    return link_time

def split_table(tableName, inter, toll, df):
    for i in range(LINK_LENGTH[tableName]):
        #把n列LINK添加到set_table_fun_new末尾
        link_name = 'link_'+str(i)
        
        #预处理数据，把有link路段的时间缺省的数据剔除
        judgeIndex = df.travel_seq.apply(lambda t:True if len(t)==LINK_LENGTH[tableName] else False)
        df = df.loc[judgeIndex]
        
        #分列
        df[str(link_name)] = df.travel_seq.apply(lambda s:float(s[i]))
    return df
        

def select_time_in_link(df):   #link级别的时间处理。最后分6个表来返回
    df['starting_time'] = df.starting_time.apply(
        per_20min)
    if {'vehicle_id','travel_time'}.issubset(df.columns):
        df = df.drop(['vehicle_id','travel_time'], axis=1)
    #set_table = pd.DataFrame(columns = ['table_A_2','table_A_3','table_B_1','table_B_3','table_C_1','table_C_3'])
    set_table_fun = {}
    set_table_fun = createTable(df)   #数据按inter_toll分成6份
    #处理travel_seq,把每个link的时间取出来
    for i in range(len(inter_toll)):
        inter, toll = inter_toll[i]
        tableName = 'table_'+str(inter)+'_'+str(toll)
        set_table_fun[tableName].travel_seq = set_table_fun[tableName].travel_seq.apply(getLink_avgTime)
        hour = set_table_fun[tableName].starting_time.dt.hour
        set_table_fun[tableName] = set_table_fun[tableName].loc[((hour >= 6) & (hour < 10)) |\
                                                                ((hour >= 15) & (hour < 19))]
        
        #暂时只取9.19之后
        d = set_table_fun[tableName].starting_time.dt.date
        set_table_fun[tableName] = set_table_fun[tableName].loc[(d >= TRAIN_START_DAY) &\
                                                                (d <= TRAIN_END_DAY)]
        
        #print(set_table_fun[tableName])
        #把set_table_fun的travel_seq拆分成n列的link
        set_table_fun[tableName] = split_table(tableName, inter, toll, set_table_fun[tableName])
        #去掉travel_seq
        set_table_fun[tableName] = set_table_fun[tableName].drop(['travel_seq'],axis=1)
    return set_table_fun
        
        

In [585]:
# 数据预处理，分别是训练集，验证集和测试集
train_df = remove_holiday(train_df)

#先按路径来分组，接着处理travel_seq字符串；然后把travel_seq拆分成n个link列
set_table_dict = select_time_in_link(train_df) 

/home/hhg/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:2387: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/home/hhg/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [586]:
print(set_table_dict['table_A_2'])  #每个link的travel_time获取正常

       intersection_id  tollgate_id       starting_time  link_0  link_1  \
59340                A            2 2016-09-19 06:00:00    8.49    4.07   
59341                A            2 2016-09-19 06:00:00   10.52    5.25   
59342                A            2 2016-09-19 06:00:00   11.15    4.34   
59347                A            2 2016-09-19 06:00:00   10.32    6.27   
59350                A            2 2016-09-19 06:20:00    9.61    4.10   
59352                A            2 2016-09-19 06:20:00    7.76    5.07   
59357                A            2 2016-09-19 06:40:00    9.31    3.88   
59358                A            2 2016-09-19 06:40:00    7.78    3.92   
59359                A            2 2016-09-19 06:40:00    9.09    4.72   
59360                A            2 2016-09-19 06:40:00    8.94    5.01   
59369                A            2 2016-09-19 06:40:00    7.82    9.58   
59371                A            2 2016-09-19 07:00:00    9.85    5.11   
59373                A   

In [587]:
def complete_miss_time(df, inter, toll, linkname, df_type='train'):
    start_day = df.starting_time.dt.date.values[0]
    end_day = df.starting_time.dt.date.values[-1]

    if df_type == 'test':
        hour_min = [(6,0), (6,20), (6,40), (7,0), (7,20), (7,40),
              (15,0), (15,20), (15,40), (16,0), (16,20), (16,40)] 
    else:        
        hour_min = [(6,0), (6,20), (6,40), (7,0), (7,20), (7,40),
                (8,0), (8,20), (8,40), (9,0), (9,20), (9,40),
              (15,0), (15,20), (15,40), (16,0), (16,20), (16,40),
              (17,0), (17,20), (17,40), (18,0), (18,20), (18,40)]
    df_comp = pd.DataFrame(columns=['intersection_id', 'tollgate_id',
                                    'starting_time',linkname])
    for d in range((end_day - start_day).days+1):
        day = start_day + timedelta(days=d)
        if ((day < NATIONNAL_START) or (day > NATIONNAL_END)):
            for j in range(len(hour_min)):
                h, m = hour_min[j]
                day_time = datetime(day.year, day.month, day.day, h, m, 0)
                index = (df.starting_time == day_time)
                avg_travel_time = df.loc[index][linkname]
                if (not avg_travel_time.empty):
                    avg = avg_travel_time.values[0]
                else:
                    avg = np.NaN
                row = {'intersection_id': inter, 'tollgate_id': toll,
                       'starting_time': str(day_time), linkname:avg} 
                df_comp = df_comp.append(row, ignore_index=True)
    
    df_comp['tollgate_id'] = df_comp['tollgate_id'].astype(int)
    df_comp.starting_time = pd.to_datetime(df_comp.starting_time)
    df_comp[linkname] = df_comp[linkname].interpolate()   #使用插值补全
    return df_comp

def create_subtable(set_table_dict):
    set_subtable_dic_fun = {}  #set_subtable_dic_fun数据组织形式 {'table_A_2':set_subtable_linktable_dic_fun,
                               #                            'table_A_3':set_subtable_linktable_dic_fun, ...}}
    for i in range(len(inter_toll)):
        inter, toll = inter_toll[i]
        tableName = 'table_'+str(inter)+'_'+str(toll)
        set_subtable_linktable_dic_fun = {} #嵌套在set_subtable_fun中的dic
        for linkOrder in range(LINK_LENGTH[tableName]):
            print('inter = ',inter,'\n','toll=',toll,'\n','linkOrder=',linkOrder,'\n')
            linktable_name = tableName + '_link_' + str(linkOrder)
            set_subtable_linktable_dic_fun[linktable_name] = set_table_dict[tableName].loc[:,\
                                                                                            ['intersection_id',\
                                                                                            'tollgate_id',\
                                                                                            'starting_time',\
                                                                                            'link_'+str(linkOrder)]]
            #groupby子表
            set_subtable_linktable_dic_fun[linktable_name] = set_subtable_linktable_dic_fun\
            [linktable_name].groupby(['intersection_id', 'tollgate_id', 'starting_time']).mean() #取中值可能更好
            set_subtable_linktable_dic_fun[linktable_name] = set_subtable_linktable_dic_fun\
            [linktable_name].reset_index()
            
            #complete_miss_time
            set_subtable_linktable_dic_fun[linktable_name] = \
            complete_miss_time(set_subtable_linktable_dic_fun[linktable_name],\
                               inter, toll, 'link_'+str(linkOrder), df_type = 'train')
            
        set_subtable_dic_fun[tableName] = set_subtable_linktable_dic_fun
    return set_subtable_dic_fun

#每个link和inter, toll, starting_time单独成一个子表，各自groupby后，进行compleme_miss_time，再合并回一个table
def groupbyProcess(set_table_dict):
    #每个link和inter, toll, starting_time单独成一个子表
    set_subtable_dic = create_subtable(set_table_dict) #输出正确

    return set_subtable_dic       
            

In [588]:
#取出相同路段的avgTime，并groupby求平均;然后interpolate
set_table_dict_afterGroupby = groupbyProcess(set_table_dict)

inter =  A 
 toll= 2 
 linkOrder= 0 

inter =  A 
 toll= 2 
 linkOrder= 1 

inter =  A 
 toll= 2 
 linkOrder= 2 

inter =  A 
 toll= 2 
 linkOrder= 3 

inter =  A 
 toll= 2 
 linkOrder= 4 

inter =  A 
 toll= 2 
 linkOrder= 5 

inter =  A 
 toll= 3 
 linkOrder= 0 

inter =  A 
 toll= 3 
 linkOrder= 1 

inter =  A 
 toll= 3 
 linkOrder= 2 

inter =  A 
 toll= 3 
 linkOrder= 3 

inter =  A 
 toll= 3 
 linkOrder= 4 

inter =  A 
 toll= 3 
 linkOrder= 5 

inter =  A 
 toll= 3 
 linkOrder= 6 

inter =  A 
 toll= 3 
 linkOrder= 7 

inter =  B 
 toll= 1 
 linkOrder= 0 

inter =  B 
 toll= 1 
 linkOrder= 1 

inter =  B 
 toll= 1 
 linkOrder= 2 

inter =  B 
 toll= 1 
 linkOrder= 3 

inter =  B 
 toll= 1 
 linkOrder= 4 

inter =  B 
 toll= 1 
 linkOrder= 5 

inter =  B 
 toll= 1 
 linkOrder= 6 

inter =  B 
 toll= 1 
 linkOrder= 7 

inter =  B 
 toll= 1 
 linkOrder= 8 

inter =  B 
 toll= 3 
 linkOrder= 0 

inter =  B 
 toll= 3 
 linkOrder= 1 

inter =  B 
 toll= 3 
 linkOrder= 2 

inter =  B 


In [589]:
#合并子表
def combineSubtable(set_table_dict_afterGroupby):
    set_table_combine_dict_fun = {}
    for i in range(len(inter_toll)):
        inter, toll = inter_toll[i]
        tableName = 'table_'+str(inter)+'_'+str(toll)
        subtable_name = tableName + '_link_'+str(0)
        set_table_combine_dict_fun[tableName] = set_table_dict_afterGroupby[tableName][subtable_name].copy()
        for linkOrder in range(1,LINK_LENGTH[tableName]):
            linkname = 'link_'+str(linkOrder)
            subtable_name = tableName + '_link_'+str(linkOrder)
            set_table_combine_dict_fun[tableName][linkname] = set_table_dict_afterGroupby[tableName][subtable_name]\
                                                                [linkname] #取列，插到新表的后面
#     print(set_table_combine_dict_fun['table_A_2'])
    return set_table_combine_dict_fun
set_table_combine_dict = combineSubtable(set_table_dict_afterGroupby)

In [593]:
print(set_table_combine_dict['table_B_1'])

    intersection_id  tollgate_id       starting_time     link_0     link_1  \
0                 B            1 2016-09-19 06:00:00  15.740000   5.000000   
1                 B            1 2016-09-19 06:20:00  13.850000  21.450000   
2                 B            1 2016-09-19 06:40:00  11.960000  37.900000   
3                 B            1 2016-09-19 07:00:00   8.315000   5.410000   
4                 B            1 2016-09-19 07:20:00  10.656667   6.320000   
5                 B            1 2016-09-19 07:40:00   9.395000   7.185000   
6                 B            1 2016-09-19 08:00:00  13.170000   6.120000   
7                 B            1 2016-09-19 08:20:00  11.167500   5.937500   
8                 B            1 2016-09-19 08:40:00   9.165000   5.755000   
9                 B            1 2016-09-19 09:00:00  10.792000   7.558000   
10                B            1 2016-09-19 09:20:00  10.868500   7.729000   
11                B            1 2016-09-19 09:40:00  10.945000 